In [4]:
#@@@@all topics addition commented to analyze difference with and without topic@@@@
import numpy as np
import pandas as p
import matplotlib.pyplot as plt
import cPickle
from sklearn.ensemble import RandomForestClassifier
documents_categories = p.read_csv("documents_categories.csv")
#clicks_test = p.read_csv("/Users/astrachan/Desktop/outbrain/clicks_test.csv", dtype={"display_id": int, "ad_id": int}))
#documents_meta = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_meta.csv")
#documents_entities = p.read_csv("/Users/astrachan/Desktop/outbrain/documents_entities.csv")
promoted_content = p.read_csv("promoted_content.csv", dtype = {"ad_id":int, "document_id":int, "campaign_id":int, "advertiser_id":int})
#sample_submission = p.read_csv("/Users/astrachan/Desktop/outbrain/sample_submission.csv")
documents_topics = p.read_csv("documents_topics.csv")
clicks_train = p.read_csv("clicks_train.csv", dtype={"display_id": int, "ad_id": int, "clicked" : bool})
events = p.read_csv("events.csv", dtype={"display_id": int, "uuid": str, "document_id" : int, "timestamp" : int, "platform" : str, "geo_location" : str})
#page_views = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views.csv")
#page_views_sample = p.read_csv("/Users/astrachan/Desktop/outbrain/page_views_sample.csv")

In [5]:
#add document to clicks_train
events.drop(events.columns[[1,3,4,5]],axis = 1,inplace=True)
clicks_train = clicks_train.merge(events, how='left', on='display_id')
del events

In [6]:
#split to validation and train, choosing display_id by random
display_ids = clicks_train.groupby(['display_id'])['display_id'].agg({'count' : 'count'}).reset_index().drop('count',axis = 1)
validation_displays = display_ids.sample(frac = 0.2)
clicks_train.reset_index(level=0, inplace=True)
validation = validation_displays.merge(clicks_train, how = 'inner', on = 'display_id')
train = clicks_train.drop(validation['index'])
del clicks_train

In [7]:
#remove all confidence levels below 0.3, can decide on another criteria later on
#documents_entities = documents_entities[documents_entities.confidence_level > 0.3]
documents_categories = documents_categories[documents_categories.confidence_level > 0.3]
#documents_topics = documents_topics[documents_topics.confidence_level > 0.3]

In [8]:
#drop confidence level
documents_categories.drop('confidence_level',axis=1, inplace=True)
#documents_topics.drop('confidence_level',axis=1, inplace=True)

In [9]:
#prepare one-hot
#topic_dummies = p.get_dummies(documents_topics['topic_id'],prefix="_ti")
categories_dummies = p.get_dummies(documents_categories['category_id'],prefix="_ci")

In [10]:
#topics = documents_topics.drop('topic_id',1).join(topic_dummies)
categories = documents_categories.drop('category_id',1).join(categories_dummies)
categories = categories.groupby(['document_id']).sum().reset_index()
#topics = topics.groupby(['document_id']).sum().reset_index()
del documents_categories, categories_dummies, # topic_dummies , documents_topics

In [11]:
#create promoted, promoted content (ad information) with its' categories and topics
promoted_content.drop(promoted_content.columns[[2,3]],axis = 1,inplace=True)
promoted = promoted_content.merge(categories, how='left', on='document_id').drop('document_id', axis=1)
#promoted = promoted.merge(topics, how='left', on='document_id')
del promoted_content
#all Nulls are of documents with no topic or category, equals to 0
promoted.fillna(0.0, inplace=True)

In [12]:
#get predictors, dummy chunk just for label names
chunk = train[:10].merge(promoted,how = 'left',on = 'ad_id')
chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
#chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
predictors=[x for x in chunk.columns if x not in ['display_id','ad_id','document_id','clicked']]

In [15]:
#number of iterations for training
print train.shape[0]/10**5

697


In [14]:
promoted

,ad_id,_ci_1000,_ci_1100,_ci_1200,_ci_1202,_ci_1203,_ci_1204,_ci_1205,_ci_1206,_ci_1207,...,_ci_1913,_ci_1914,_ci_1915,_ci_2000,_ci_2002,_ci_2003,_ci_2004,_ci_2005,_ci_2006,_ci_2100
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#initiallize algorithm then fit it on chunks of size 10**5 - takes an hour or so 
#@@much more!, it's because the n_estimators +=3
#might be a problem with the n_estimators growing constantly, 
#usually people stop the growing when the algorithm reaches good prediction
alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
i = 0
k = 2
for chunk in np.array_split(train, train.shape[0]/10**5):
    alg.n_estimators += k
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    #chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    alg.fit(chunk[predictors], chunk["clicked"])
    i += 1
    if i % 50 == 0:
        if i == 50:
            k = 1
        else:
            k = 0
    print i

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


C:\Users\rludmer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\ensemble\forest.py:267: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350


In [19]:
#save outputted tree with name [tree_name]
tree_name = 'fourthTree_n_est_drop_no_topics'
with open(tree_name, 'wb') as f:
    cPickle.dump(alg, f)

In [20]:
#number of iterations for validating
print (validation.shape[0]/10**5)

174


In [21]:
#run outputted tree on chunks of size 10**5 of validation
predY=[]
i = 0
for chunk in np.array_split(validation, validation.shape[0]/10**5):
    chunk = chunk.merge(promoted,how = 'left',on = 'ad_id')
    chunk = chunk.merge(categories, how='left', on='document_id',suffixes=('_ad', '_doc'))
    #chunk = chunk.merge(topics, how='left', on='document_id',suffixes=('_ad', '_doc'))
    chunk.fillna(0.0, inplace=True)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred
    i += 1
    print i

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


In [22]:
predict = np.asarray(predY)
val_copy = validation.copy()
val_copy['predict'] = predict
#delete predict? validation? chunk? chunk_pred? predictors? predY? bentaim lo

In [27]:
#function evaluating how far were you from predicting right
val_copy.sort_values(['display_id', 'predict'], inplace=True, ascending=[True, False] )
val_copy["seq"] = np.arange(val_copy.shape[0])
Y_seq = val_copy[val_copy.clicked == 1].seq.values
Y_first = val_copy[['display_id', 'seq']].drop_duplicates(subset='display_id', keep='first').seq.values
Y_ranks = Y_seq - Y_first
score = np.mean(1.0 / (1.0 + Y_ranks))
print("MAP: %.12f" % score)

MAP: 0.531798825681


In [28]:
val_copy

,display_id,index,ad_id,clicked,document_id,predict,true_and_true,seq
5531721,3,12,152141,False,1179111,0.296756,0,0
5531722,3,13,183846,False,1179111,0.296756,0,1
5531720,3,11,95814,False,1179111,0.274072,0,2
5531723,3,14,228657,True,1179111,0.220339,0,3
5531719,3,10,71547,False,1179111,0.150425,0,4
5531724,3,15,250082,False,1179111,0.150425,0,5
17306592,14,62,224171,True,1759953,0.329125,0,6
17306591,14,61,143467,False,1759953,0.300947,0,7
17306590,14,60,98270,False,1759953,0.223002,0,8
17306593,14,63,288396,False,1759953,0.140864,0,9


In [29]:
max_rows = val_copy.groupby(['display_id'])['predict'].transform(max) == val_copy['predict']

In [30]:
final = val_copy[max_rows]

In [32]:
success = final[final['clicked'] == True]

In [33]:
success

,display_id,index,ad_id,clicked,document_id,predict,true_and_true,seq
17306592,14,62,224171,True,1759953,0.329125,0,6
16282038,16,71,273567,True,1727882,0.267726,0,10
8506142,37,167,234713,True,1779285,0.304805,0,26
8451910,54,260,257105,True,1060089,0.311341,0,39
10389108,61,297,77848,True,1282043,0.268559,0,48
7138301,91,437,145937,True,1179111,0.292874,0,59
15371924,115,565,198400,True,1789106,0.311341,0,97
1512254,138,691,195273,True,1782482,0.312061,0,124
190088,153,767,137758,True,1778648,0.265464,0,134
10936517,156,784,97177,True,313956,0.293410,0,147


In [34]:
float(len(success)) / float(len(final))

0.26449683597087525